In [8]:
%matplotlib auto  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor, Slider, CheckButtons, TextBox

Using matplotlib backend: Qt5Agg


In [9]:
disrupted_list = ["CNK US Equity", "CGX CN Equity", "7203 JP Equity", "6758 JP Equity", "7974 JP Equity", 
                  "WPP LN Equity", "OMC US Equity", "SAP GY Equity", "ORCL US Equity", "NJOB SS Equity", 
                  "WAGE US Equity", "WMT US Equity", "CVS US Equity", "UBSGE SW Equity", "DBK GY Equity", 
                  "SAN SM Equity", "ANZ AU Equity", "JNPR US Equity", "AEO US Equity", "ANF US Equity", 
                  "GPS US Equity", "KSS US Equity", "GPC US Equity", "KR US Equity", "COST US Equity", 
                  "M US Equity", "BBY US Equity", "PAYX US Equity", "CMCSA US Equity", "TGT US Equity", 
                  "AMC US Equity", "CINE US Equity", "SPX Index"]

#Cineworld, Cinemark, Cineplex, AMC Entertainment
cinema_list = ["CINE LN Equity", "CNK US Equity", "CGX CN Equity", "AMC US Equity"]

#Totota
car_list = ["7203 JP Equity"]

#Comcast
multimedia_list = ["CMCSA US Equity"]

#Sony, Nintendo
game_list = ["6758 JP Equity", "7974 JP Equity"]

#WPP, Omnicom
mediaAd_list = ["WPP LN Equity", "OMC US Equity"]

#SAP, Oracle, Juniper Networks
software_list = ["SAP GY Equity", "ORCL US Equity", "JNPR US Equity"]

#Netjobs, Wagework, Paychex
employment_list = ["NJOB SS Equity", "WAGE US Equity", "PAYX US Equity"]

#Walmart, CVS Health, Kroger, Costco, Macy's Inc, Bestbuy, Target
retail_list = ["WMT US Equity", "CVS US Equity", "KR US Equity", "COST US Equity", 
               "M US Equity", "BBY US Equity", "TGT US Equity"]

#American Eagle Outfitters, Abercrombie & Fitch, GAP, Kohl, 
clothRetail_list = ["AEO US Equity", "ANF US Equity", "GPS US Equity", "KSS US Equity"]

#Genuine Parts
partRetail_list = ["GPC US Equity"]

#UBS Group AG, Deutsche Bank, Banco Santander SA, ANZ
bank_list = ["UBSGE SW Equity", "DBK GY Equity", "SAN SM Equity", "ANZ AU Equity"]

sp500_list = ["SPX Index"]


sector_list = [cinema_list]
sector_list.append(car_list)
sector_list.append(multimedia_list)
sector_list.append(game_list)
sector_list.append(mediaAd_list)
sector_list.append(software_list)
sector_list.append(employment_list)
sector_list.append(retail_list)
sector_list.append(clothRetail_list)
sector_list.append(partRetail_list)
sector_list.append(bank_list)
#sector_list.append(sp500_list)

print("Disrupted list length = {}".format(len(disrupted_list)))
print("Sector list length = {}".format(sum(len(a) for a in sector_list)))


crisis_2008 = ["2008-05-01", "2009-12-31"]
china_2016 = ["2015-12-1", "2016-02-29"]
greek_2015 = ["2015-07-1", "2015-09-30"]
faebook_2018 = ["2018-01-01", "2018-03-31"]

Disrupted list length = 33
Sector list length = 32


In [10]:
monthlyData_df = pd.read_csv("data.csv", index_col=0)

#Make sure the index is in date format
monthlyData_df.index = pd.to_datetime(monthlyData_df.index)

#Fill all nan with 0
monthlyData_df = monthlyData_df.fillna(0)

print(monthlyData_df.shape)

(352, 57)


In [11]:
#Calculate monthly return
monthlyReturn_df = pd.DataFrame(index=monthlyData_df.index)
for stockCode in monthlyData_df:        
    monthlyReturn_df[stockCode] = monthlyData_df[stockCode].pct_change(-1)+1

#Replace inf with nan    
monthlyReturn_df = monthlyReturn_df.replace(np.inf, np.nan)
print(monthlyReturn_df.head())

            AMZN US Equity  MSFT US Equity  CRM US Equity  NOW US Equity  \
Dates                                                                      
2019-04-30        1.081859        1.107343       1.044074       1.101505   
2019-03-29        1.085936        1.052754       0.967736       1.029444   
2019-02-28        0.954094        1.072776       1.076857       1.088265   
2019-01-31        1.144317        1.028158       1.109513       1.235720   
2018-12-31        0.888650        0.915953       0.959442       0.961030   

            WDAY US Equity  DOCU US Equity  GOOGL US Equity  ADBE US Equity  \
Dates                                                                         
2019-04-30        1.066269        1.093171         1.018753        1.085407   
2019-03-29        0.974334        0.939982         1.044685        1.015200   
2019-02-28        1.090343        1.115268         1.000586        1.059237   
2019-01-31        1.136836        1.233782         1.077448        1.095

In [12]:
'''
Calculate the total return based on start time, end time, return and ticker list
'''
def calculateReturn(startTime, endTime, return_df, ticker_list):
    data_df = return_df[ticker_list]
    data_filter_df = data_df.loc[data_df.index >= startTime]
    data_filter_df = data_filter_df.loc[data_filter_df.index <= endTime]
    
    #https://stackoverflow.com/questions/33750326/compute-row-average-in-pandas
    #Compute row average
    data_filter_df = data_filter_df.apply(lambda row: row.fillna(row.mean()), axis=1)

    #With the row average, calculate the mean return month by month
    data_mean_df = data_filter_df.mean(axis=1)

    #calculate the total return using the mean return
    data_return = data_mean_df.product(axis=0) - 1    
    return data_return
    
startTime = "2009-01-01"
endTime = "2014-12-31"

    
retail_return = calculateReturn(startTime, endTime, monthlyReturn_df, retail_list)
clothRetail_return = calculateReturn(startTime, endTime, monthlyReturn_df, clothRetail_list)
bank_return = calculateReturn(startTime, endTime, monthlyReturn_df, bank_list)

fig = plt.figure(figsize=(8, 6))

#"111" means "1x1 grid, first subplot" and "234" means "2x3 grid, 4th subplot".
#Need subplot for cursor
barSubPlot = fig.add_subplot(111, facecolor='#FFFFCC')

#adjust the bottom of the subplot a bit, to avoid overlap
#Bottom = 0.25 mean move the bottom to the top by 25%
plt.subplots_adjust(bottom=0.25)

#The list contain the value for bar
data = []    

calculate_list = sector_list.copy() #True copy here
    
for sector in calculate_list:        
    myReturn = calculateReturn(startTime, endTime, monthlyReturn_df, sector)
    data.append(myReturn)
    
#Will always draw S&P500 last
data.append(calculateReturn(startTime, endTime, monthlyReturn_df, sp500_list))
    
#Index for the x axes, should not change
ind = np.arange(len(data))

#Color of bar
barColor = (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)
barPlot = barSubPlot.bar(ind, data, width=0.35, color=barColor)

label_list = ["Cinema", "Car", "Multimedia", "Game", "Ad", "Software", "Employment", "Retail", "Clothes", "Parts", 
              "Bank"]

myLabel = label_list.copy()
myLabel.append("S&P500")

plt.xticks(ind, myLabel)
plt.yticks(np.arange(0, 2, 0.10))

# set useblit = True on gtkagg for enhanced performance
cursor = Cursor(barSubPlot, useblit=True, color='red', linewidth=0.5)

axcolor = 'lightgoldenrodyellow'
#4-tuple of floats rect = [left, bottom, width, height]. A new axes is added with dimensions rect 
#in normalized (0, 1) units using add_axes on the current figure.
axStartYear = plt.axes([0.15, 0.15, 0.65, 0.03], facecolor=axcolor)
axEndYear = plt.axes([0.15, 0.1, 0.65, 0.03], facecolor=axcolor)
axCategory = plt.axes([0.01, 0.4, 0.09, 0.5])

startYear_float = 2000
endYear_float = 2000 + (2020-2000)*12

startYear_slider = Slider(axStartYear, 'Start Year', startYear_float, endYear_float, valinit=2000, valfmt='%.0f-01-01')
endYear_slider = Slider(axEndYear, 'End Year', startYear_float, endYear_float, valinit=endYear_float, valfmt='%.0f-01-31')
check_button = CheckButtons(axCategory, label_list, [True]*len(label_list))

isIncluded_list = [True]*len(label_list)

#Function to update the draw
def update(label):    
    global startTime, endTime, calculate_list, barPlot, myLabel
    
    if label in label_list:
        labelIndex = label_list.index(label)
        isIncluded_list[labelIndex] = not isIncluded_list[labelIndex]
        calculate_list = []                
        myLabel = []
        
        #Add sectors to the list to be calculate return
        for myIndex, item in enumerate(isIncluded_list):
            if(item):                
                calculate_list.append(sector_list[myIndex])
                myLabel.append(label_list[myIndex])
        
        #Will always draw S&P500 last
        myLabel.append("S&P500")

    startYear_int = round(startYear_slider.val,0)    
    startYear_float = 2000 + (startYear_int-2000)//12
    startMonth_float = (startYear_int-2000)%12 + 1
    
    endYear_int = round(endYear_slider.val,0)        
    endYear_float = 2000 + (endYear_int-2000)//12
    endMonth_float = (endYear_int-2000)%12 + 1
    
    #Since the slider return float, need to convert to string 
    newStartYear = '%0.0f' % startYear_float
    newEndYear = '%0.0f' % endYear_float
    
    newStartMonth = '%02.0f' % startMonth_float
    newEndMonth = '%02.0f' % endMonth_float    
    
    #Replace old year with new year:
    startTime = newStartYear + "-" + newStartMonth + "-01"
    endTime = newEndYear + "-" + newEndMonth + "-01"
    
    startYear_slider.valtext.set_text(startTime)
    endYear_slider.valtext.set_text(endTime)
    
    
    data = []    
    
    for sector in calculate_list:        
        myReturn = calculateReturn(startTime, endTime, monthlyReturn_df, sector)
        data.append(myReturn)
    
    #Will always draw S&P500 last
    data.append(calculateReturn(startTime, endTime, monthlyReturn_df, sp500_list))
    
    #Index for the x axes, should not change
    ind = np.arange(len(data))
    
    barPlot.remove()
    barPlot = barSubPlot.bar(ind, data, width=0.35, color=barColor)
    barPlot[-1].set_color('r')
    
    # https://stackoverflow.com/questions/43673884/change-x-axis-ticks-to-custom-strings
    # Set the tick positions
    barSubPlot.set_xticks(ind)    
        
    # Set the tick labels
    barSubPlot.set_xticklabels(myLabel)
    
    #print(plt.xticks()[1][0])
    #Set the ticks freq
    plt.setp(barSubPlot, yticks = (np.min(data), np.max(data), (np.max(data)-np.min(data))/10))
    #If the limit was higher then they can't scale back, need to set the y limit as well
    barSubPlot.set_ylim(bottom=(np.min(data)-0.2), top=(np.max(data)+0.2))
    
    #This is for setting the height of bars
    for rect, h in zip(barPlot, data):
        rect.set_height(h)    
    
    #Remove all old text    
    #If you remove it in the barSubPlot.texts, some how it mess up with the iteration :-/
    #For example, if you remove texts[0], then texts[1] will immediately replace text[0] as text[0]
    #So you can't remove it with the loop
    #Best to move it to another list and then remove it from there
    text_list = []    
    for text in barSubPlot.texts:
        text_list.append(text)       
    
    for text in text_list:
        text.remove()
            
  
    for i, value in enumerate(data):
        #print("{0:2d}: {1}".format(i, value))
        barSubPlot.text(i, value+0.05, "{0:.1f}%".format(value*100), color='blue')        
 
startYear_slider.on_changed(update)
endYear_slider.on_changed(update)
check_button.on_clicked(update)


0

In [13]:
data_df = monthlyReturn_df[bank_list]
startTime = "2009-01-01"
endTime = "2014-12-31"
data_filter_df = data_df.loc[data_df.index >= startTime]
data_filter_df = data_filter_df.loc[data_filter_df.index <= endTime]
#print(data_filter_df)

data_return = data_filter_df.product(axis=0) - 1    
print(data_return)

UBSGE SW Equity    0.000000
DBK GY Equity     -0.111471
SAN SM Equity     -0.111448
ANZ AU Equity      1.431716
dtype: float64


In [14]:
print(monthlyReturn_df[sp500_list].where(monthlyReturn_df[sp500_list]<0.96).dropna())

            SPX Index
Dates                
2018-12-31   0.908223
2018-10-31   0.930597
2016-01-29   0.949265
2015-08-31   0.937419
2012-05-31   0.937349
2011-09-30   0.928238
2011-08-31   0.943209
2010-08-31   0.952551
2010-06-30   0.946118
2010-05-31   0.918024
2009-02-27   0.890069
2009-01-30   0.914343
2008-11-28   0.925151
2008-10-31   0.830575
2008-09-30   0.909209
2008-06-30   0.914038
2008-01-31   0.938837
2007-11-30   0.955957
2002-12-31   0.939667
2002-09-30   0.889976
2002-07-31   0.921005
2002-06-28   0.927535
2002-04-30   0.938582
2001-09-28   0.918277
2001-08-31   0.935892
2001-03-30   0.935795
2001-02-28   0.907709
2000-11-30   0.919931
2000-09-29   0.946517
2000-01-31   0.949096
1998-08-31   0.854203
1997-08-29   0.942554
1997-03-31   0.957386
1996-07-31   0.954252
1994-03-31   0.954254
1991-11-29   0.956072
1991-06-28   0.952107
1990-09-28   0.948816
1990-08-31   0.905686
